# Extract the Personal Identifiable Information (PII) using Watson NLP

<h2>Use Case</h2>

This notebook demonstrates how to extract PII entities using Watson NLP Pertained models also demonstrates how to prepare custom train models. PII (Personally extraction is the process of identifying and extracting personal information from a document or dataset. This information can include names, addresses, phone numbers, email addresses, Social Security numbers, Credit Card number, and other types of information that can be used to identify an individual. 


<h2>What you'll learn in this notebook</h2>

Watson NLP offers Pertained Models for various NLP tasks also provides fine-tune functionality for custom training. This notebooks shows:

* <b>RBR</b>:  A Rule-Based Reasoner (RBR) in NLP works by using a set of predefined rules to process and understand natural language input. These rules are used to identify specific patterns or structures in the input text and determine the meaning of the text based on those patterns.


* <b>BILSTM</b>: the BiLSTM network would take the preprocessed text as input and learn to identify patterns and relationships between words that are indicative of PII data. The BiLSTM network would then output a probability score for each word in the text, indicating the likelihood that the word is part of a PII entity. The BiLSTM network may also be trained to recognize specific entities such as names, addresses, phone numbers, email addresses, etc.


* <b>SIRE</b>: Statistical Information and Relation Extraction (SIRE) is a technique used in natural language processing (NLP) to extract specific information and relationships from text. It involves using machine learning algorithms to identify and extract structured data such as entities, attributes, and relations from unstructured text. SIRE is used in a variety of applications, including information extraction, knowledge graph construction, and question answering. SIRE typically uses supervised learning approach, where a model is trained using annotated examples of text and the corresponding structured data. The model can then be used to extract the same information from new, unseen text.


* <b>Bert</b>: IBM Watson NLP BERT uses a pre-trained version of BERT that was trained on a large corpus of text data. The pre-trained model can be fine-tuned on a specific task such as text classification, named entity recognition, and more. The BERT architecture consists of an encoder network that is made up of multiple layers of transformer blocks. Each transformer block includes a self-attention mechanism and a feed-forward neural network.


* <b>Transformer</b>: This model is a neural network architecture that is used in natural language processing tasks such as language translation, text summarization, and language generation. It is based on self-attention mechanism and can be used to extract information such as named entities, relationships and sentiments from the text.

## Table of Contents


1.  [Before you start](#beforeYouStart)
1.	[Load Entity PII Models](#LoadModel)
1.  [Watson NLP Models](#NLPModels)    
    1. [BiLSTM Pretrained](#BILSTMPre)
    1. [RBR Pretrained](#RBRPre)
    
1.  [Testing With Hanzo's Test Dataset](#Testing)    
1.  [Summary](#summary)

<a id="beforeYouStart"></a>
### 1. Before you start


<div class="alert alert-block alert-danger">
<b>Stop kernel of other notebooks.</b></div>

**Note:** If you have other notebooks currently running with the _Default Python 3.8 + Watson NLP XS_ environment, **stop their kernels** before running this notebook. All these notebooks share the same runtime environment, and if they are running in parallel, you may encounter memory issues. To stop the kernel of another notebook, open that notebook, and select _File > Stop Kernel_.

<div class="alert alert-block alert-warning">
<b>Set Project token.</b></div>

Before you can begin working on this notebook in Watson Studio in Cloud Pak for Data as a Service, you need to ensure that the project token is set so that you can access the project assets via the notebook.

When this notebook is added to the project, a project access token should be inserted at the top of the notebook in a code cell. If you do not see the cell above, add the token to the notebook by clicking **More > Insert project token** from the notebook action bar.  By running the inserted hidden code cell, a project object is created that you can use to access project resources.

![ws-project.mov](https://media.giphy.com/media/jSVxX2spqwWF9unYrs/giphy.gif)

<div class="alert alert-block alert-info">
<b>Tip:</b> Cell execution</div>

Note that you can step through the notebook execution cell by cell, by selecting Shift-Enter. Or you can execute the entire notebook by selecting **Cell -> Run All** from the menu.

In [2]:
import json
import pandas as pd
import watson_nlp
from watson_nlp import data_model as dm
from watson_nlp.toolkit.entity_mentions_utils import prepare_train_from_json

In [3]:
# Silence Tensorflow warnings
import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.autograph.set_verbosity(0)

<a id="LoadModel"></a>
### 2. Load Entity PII Models

In [4]:
# Load a syntax model to split the text into sentences and tokens
syntax_model = watson_nlp.load(watson_nlp.download('syntax_izumo_en_stock'))
# Load bilstm model in WatsonNLP
bilstm_model = watson_nlp.load(watson_nlp.download('entity-mentions_bilstm_en_pii'))
# Load rbr model in WatsonNLP
rbr_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_multi_pii'))
# Download the GloVe model to be used as embeddings in the BiLSTM
glove_model = watson_nlp.load(watson_nlp.download('embedding_glove_en_stock'))
# Download the algorithm template
mentions_train_template = watson_nlp.load(watson_nlp.download('file_path_entity-mentions_sire_multi_template-crf'))
# Download the feature extractor
default_feature_extractor = watson_nlp.load(watson_nlp.download('feature-extractor_rbr_entity-mentions_sire_en_stock'))
# Download and load the pretrained model resource
#pretrained_model_resource = watson_nlp.load(watson_nlp.download('pretrained-model_roberta-base_v2-8-0_llm_transformer_lang_en_cased_2022-05-06-052653'))

In [11]:
text = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][1], df['Credit Card Number'][1])

<a id="NLPModels"></a>
### 3.  Watson NLP Models

<a id="BILSTMPre"></a>
### BiLSTM Pretrained

In [50]:
#help(bilstm_model)

In [14]:
syntax_result = syntax_model.run(text)
bilstm_result = bilstm_model.run(syntax_result)
bilstm_result

{
  "mentions": [
    {
      "span": {
        "begin": 11,
        "end": 24,
        "text": "Robert Aragon"
      },
      "type": "Person",
      "producer_id": {
        "name": "BiLSTM Entity Mentions",
        "version": "1.0.0"
      },
      "confidence": 0.8697594404220581,
      "mention_type": "MENTT_UNSET",
      "mention_class": "MENTC_UNSET",
      "role": ""
    }
  ],
  "producer_id": {
    "name": "BiLSTM Entity Mentions",
    "version": "1.0.0"
  }
}

<a id="RBRPre"></a>

### RBR Pretrained


In [52]:
text1 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][1], "` 378282246310005 `")
text2 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][2], "`378282246310005 `")
text3 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][3], "` 378282246310005 `")
text4 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][4], "` 5555555555554444 `")
text5 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][5], "`5555555555554444 `")
text6= "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][6], "` 5555-5555-5555-4444 `")
text7 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][7], "`5555-5555-5555-4444 `")
text8 = "My name is %s, and my social security number is %s. Here's the number to my Visa credit card, %s" % (df['First and Last Name'][1], df['SSN'][8], "` 5555-5555-5555-4444`")

In [53]:
text1

"My name is Robert\xa0Aragon, and my social security number is 489-36-8350. Here's the number to my Visa credit card, ` 378282246310005 `"

In [54]:
all_test=[text1,text2,text3,text4,text5,text6,text7,text8]

In [ ]:
#help(rbr_model)

In [55]:
# Test the pretrain
for test in all_test:
    rbr_result = rbr_model.run(test, language_code='en')

    for i in rbr_result.mentions:
        print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  514-14-8905     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  690-05-5315     Type:  NationalNumber.SocialSecurityNumber.US
Text:  378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text:  421-37-1396     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  458-02-6124     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  612-20-6832     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  300-62-3266     Type:  NationalNumber.SocialSecurityNumber.US
Text:  5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master
Text:  66

In [29]:
#test dataset for URL extraction 

test1 = "My name is Robert\xa0Aragon, and my social security number is 489-36-8350. Here's the number to my Visa credit card, http://www.example.com/page_id=5555555555554444"

<a id="Testing"></a>

## 4. Testing With Hanzo's Test Dataset

### RBR Stock (URL Extraction)

In [30]:
#Test Pretrained rbr stock model in WatsonNLP
rbr_ent_model = watson_nlp.load(watson_nlp.download('entity-mentions_rbr_en_stock'))
rbr_ent_result = rbr_ent_model.run(test1)

for i in rbr_ent_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

Text:  489-36-8350     Type:  PhoneNumber
Text:  http://www.example.com/page_id=5555555555554444 Type:  URL


In [31]:
#Test Pretrained rbr PII model in WatsonNLP
test2 = "http://www.example.com/page_id=5555555555554444"

rbr_result_pii = rbr_model.run(test2, language_code='en')
rbr_result_pii

for i in rbr_result_pii.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

In [30]:
#Test Pretrained bilstm_model model in WatsonNLP
syntax_result = syntax_model.run(test2)
bilstm_result = bilstm_model.run(syntax_result)
bilstm_result

for i in bilstm_result.mentions:
    print("Text: ", i.span.text.ljust(15, " "), "Type: ", i.type)

In [18]:
# Testing dataset
text1 = "Here's the number to my Visa credit card, %s" % ( "378282246310005")
text2 = "Here's the number to my Visa credit card, %s" % ( " 378282246310005")
text3 = "Here's the number to my Visa credit card, %s" % ( "378282246310005 ")
text4 = "Here's the number to my Visa credit card, %s" % ( " 378282246310005 ")
text5 = "Here's the number to my Visa credit card, %s" % ( "5555555555554444")
text6 = "Here's the number to my Visa credit card, %s" % ( " 5555555555554444")
text7 = "Here's the number to my Visa credit card, %s" % ( "5555555555554444 ")
text8 = "Here's the number to my Visa credit card, %s" % ( " 5555555555554444 ")
text9= "Here's the number to my Visa credit card, %s" % ( "5555-5555-5555-4444")
text10 = "Here's the number to my Visa credit card, %s" % ( " 5555-5555-5555-4444")
text11= "Here's the number to my Visa credit card, %s" % ( "5555-5555-5555-4444 ")
text12 = "Here's the number to my Visa credit card, %s" % ( " 5555-5555-5555-4444 ")

all_test=[text1,text2,text3,text4,text5,text6,text7,text8,text8,text9,text10,text11,text12]

In [25]:
# Test the pretrain
t=0
for test in all_test:
    rbr_result = rbr_model.run(test, language_code='en')
    
    for i in rbr_result.mentions:
        print("Text"+str(t), i.span.text.ljust(15, " "), "Type: ", i.type)
    t+=1

Text2 378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text3 378282246310005 Type:  BankAccountNumber.CreditCardNumber.Amex
Text6 5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text7 5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text8 5555555555554444 Type:  BankAccountNumber.CreditCardNumber.Master
Text9 5555-5555-5555  Type:  PhoneNumber
Text10 5555-5555-5555  Type:  PhoneNumber
Text11 5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master
Text12 5555-5555-5555-4444 Type:  BankAccountNumber.CreditCardNumber.Master


<a id="summary"></a>
## 5. Summary

<span style="color:blue">This notebook shows you how to use the Watson NLP library to:
1. Extract PII using Pertained Models
</span>

Please note that this content is made available by IBM Build Lab to foster Embedded AI technology adoption. The content may include systems & methods pending patent with USPTO and protected under US Patent Laws. For redistribution of this content, IBM will use release process. For any questions please log an issue in the GitHub.

Developed by IBM Build Lab

Copyright - 2022 IBM Corporation